In [3]:
# Imports
import pandas as pd
import os 
from bs4 import BeautifulSoup as bs
from splinter import Browser
import traceback
from time import sleep

In [4]:
path= os.path.join('ChromeDriver','chromedriver.exe')
executable_path = {'executable_path':path}

browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# Visit the following URL
url = "https://stats.nba.com/"
browser.visit(url)

In [6]:
#Navigate through website the get the stats table of best players (points) from seasons 2000 to 2010

browser.click_link_by_partial_text('See all Player Stats')
sleep(1)
html = browser.html
soup = bs(html, 'html.parser')

element = browser.find_by_name('Season').first
element.select('object:27')

D:\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,
